**Table of contents**<a id='toc0_'></a>    
- [N2 Excited State Data](#toc1_)    
- [NH3 Excited States](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[N2 Excited State Data](#toc0_)



just use pandoc style citations [@haupt2022; @laughlinTheory2000] and worry about matching citations when building in LaTeX (use pandoc to convert these)

!cat table.md


In [1]:
from IPython.display import Markdown, display
import pandas as pd
import uncertainties
from uncertainties import ufloat_fromstr
import string

display_csv = lambda f: display(Markdown(pd.read_csv(f,comment="#").to_markdown(index=False)))
display_md = lambda f: display(Markdown(open(f).read()))

In [4]:
import pandas as pd

from uncertainties import unumpy
from uncertainties import ufloat, ufloat_fromstr
from uncertainties.core import UFloat
from math import isnan

def format_ufloat(x):
    if isinstance(x, type(ufloat(1, 1))):
        return f"{x:+.1uS}"  # Uses the shorthand format
    return x

def col2ufloat(col):
    return [ufloat_fromstr("nan") if pd.isna(x) else ufloat_fromstr(x) for x in col]

def display_formatted(df):
    # Create a copy to avoid modifying original
    display_df = df.copy()

    # Format ufloat columns
    for col in df.columns:
        if isinstance(df[col].iloc[0], UFloat):
            display_df[col] = df[col].apply(lambda x: f"{x:+.1uS}")

    display(display_df)

def get_state_values(filename):
    # read in data
    df = pd.read_csv(filename, comment='#', skipinitialspace=True)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
    df.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too

    # handle NaNs as ufloats as well
    df["avdz"] = col2ufloat(df["avdz"])
    df["avtz"] = col2ufloat(df["avtz"])

    if "avqz" in df.columns: df["avqz"] = col2ufloat(df["avqz"])

    display_formatted(df)

    # Get gs values as a series
    gs_values = df[df['state'] == 'gs'].iloc[0]
    # display(gs_values)

    # Create new dataframe excluding gs
    diff_df = df[df['state'] != 'gs'].copy()

    cols_to_diff = df.columns[1:]  # All columns except first
    for col in cols_to_diff:
        # display(diff_df[col])
        # display(gs_values[col])
        diff_df[col] = diff_df[col] - gs_values[col]
    for col in diff_df.columns[1:]:
        diff_df[col] *= 1000
    display_formatted(diff_df)

    return df, diff_df
    # type(df["avdz"][1])

In [5]:
df = pd.read_csv("N2_casci_states.csv", comment='#', skipinitialspace=True)

display(df)

,state,avdz,avtz
0,gs,-109.4869509(85),-109.535352(13)
1,1Pig,-109.145095(39),-109.195608(166)
2,1Sigu-,-109.120941(61),-109.173683(276)
3,1Delu,-109.107147(35),-109.159330(295)
4,3Sigu+,-109.201530(12),-109.250179(45)
5,3Pig,-109.192064(10),-109.239100(26)
6,3Delu,-109.156060(8),-109.207600(27)


In [6]:
df_casscf, dfdiff_casscf = get_state_values("N2_casscf_states.csv")
df_casci, dfdiff_casci = get_state_values("N2_casci_states.csv")

,state,avdz,avtz,avqz
0,gs,-109.478083(2),-109.53226(2),-109.54163(2)
1,1Pig,-109.137355(3),-109.19071(4),-109.1997(2)
2,1Sigu-,-109.113289(3),-109.16967(9),+nan(1)
3,1Delu,-109.100518(3),-109.1555(1),+nan(1)
4,3Sigu+,-109.195402(2),-109.24817(2),+nan(1)
5,3Pig,-109.185292(4),-109.23768(1),+nan(1)
6,3Delu,-109.152245(3),-109.20649(2),+nan(1)


,state,avdz,avtz,avqz
1,1Pig,+340.729(3),+341.56(4),+341.9(2)
2,1Sigu-,+364.794(3),+362.60(9),+nan(1)e+03
3,1Delu,+377.565(3),+376.8(1),+nan(1)e+03
4,3Sigu+,+282.681(3),+284.10(3),+nan(1)e+03
5,3Pig,+292.791(4),+294.59(2),+nan(1)e+03
6,3Delu,+325.839(3),+325.77(3),+nan(1)e+03


,state,avdz,avtz
0,gs,-109.486951(8),-109.53535(1)
1,1Pig,-109.14509(4),-109.1956(2)
2,1Sigu-,-109.12094(6),-109.1737(3)
3,1Delu,-109.10715(3),-109.1593(3)
4,3Sigu+,-109.20153(1),-109.25018(5)
5,3Pig,-109.19206(1),-109.23910(3)
6,3Delu,-109.156060(8),-109.20760(3)


,state,avdz,avtz
1,1Pig,+341.86(4),+339.7(2)
2,1Sigu-,+366.01(6),+361.7(3)
3,1Delu,+379.80(4),+376.0(3)
4,3Sigu+,+285.42(1),+285.17(5)
5,3Pig,+294.89(1),+296.25(3)
6,3Delu,+330.89(1),+327.75(3)


In [5]:
df_exp = pd.read_csv("N2_benchmark.csv", comment='#', skipinitialspace=True)
df_exp = df_exp.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
df_exp.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too
df_exp["exp1"] *= 37
df_exp["exp2"] *= 37
df_exp["theory"] *= 37
# df.to_csv("N2_benchmark_converted.csv", index=False)

In [6]:
# seem to struggle with singlets but not triplets
print("N2 Loos")
display_csv("N2_loos.csv")

print("N2 CASSCF")
display_formatted(dfdiff_casscf)

print("N2 CASCI")
display_formatted(dfdiff_casci)

print("N2 Exp")
display_formatted(df_exp)

N2 Loos


|  state     |  avdz        |  avtz        |  avqz    |
|:-----------|:-------------|:-------------|:---------|
| 1Pig       | 345.8(0)     | 343.2(0)     | 343.2(0) |
| 1Sigu-     | 369.3(0)     | 363.1(0)     | 364.6(0) |
| 1Delu      | 383.3(0)     | 378.2(0)     | 378.9(0) |
| 3Sigu+     | 283.0(0)     | 283.0(0)     | 284.4(0) |
| 3Pig       | 295.8(0)     | 294.4(0)     | 295.1(0) |
| 3Delu      | 329.3(0)     | 326.0(0)     | 326.3(0) |

N2 CASSCF


,state,avdz,avtz
1,1Pig,+340.729(3),+341.56(4)
2,1Sigu-,+364.794(3),+362.60(9)
3,1Delu,+377.565(3),+376.8(1)
4,3Sigu+,+282.681(3),+284.10(3)
5,3Pig,+292.791(4),+294.59(2)
6,3Delu,+325.839(3),+325.77(3)


N2 CASCI


,state,avdz,avtz
1,1Pig,+341.86(4),+339.7(2)
2,1Sigu-,+366.01(6),+361.7(3)
3,1Delu,+379.80(4),+376.0(3)
4,3Sigu+,+285.42(1),+285.17(5)
5,3Pig,+294.89(1),+296.25(3)
6,3Delu,+330.89(1),+327.75(3)


N2 Exp


,state,exp1,exp2,theory
0,1Pig,344.47,344.47,342.99
1,1Sigu-,367.04,367.04,373.33
2,1Delu,379.99,379.99,389.98
3,3Sigu+,286.75,286.75,279.72
4,3Pig,297.48,297.48,297.85
5,3Delu,328.56,328.56,330.41


In [7]:
dfdiff_casscf.to_latex("N2_casscf_diff.tex", index=False)
dfdiff_casci.to_latex("N2_casci_diff.tex", index=False)

# <a id='toc2_'></a>[NH3 Excited States](#toc0_)

In [13]:
df_exp = pd.read_csv("NH3_benchmark.csv", comment='#', skipinitialspace=True)
df_exp = df_exp.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
df_exp.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too
df_exp["exp1"] *= 37
df_exp["exp2"] *= 37
df_exp["theory"] *= 37
display_formatted(df_exp)
df.to_csv("NH3_benchmark_converted.csv", index=False)

,state,exp1,exp2,theory
0,1E,292.30,293.41,296.74
1,3A2,222.74,NaN,NaN


In [14]:
df_loos = pd.read_csv("NH3_loos.csv", comment='#', skipinitialspace=True)
df_loos = df_loos.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
df_loos.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too
df_loos["avdz"]
df_loos["avdz"] *= 37
df_loos["avtz"] *= 37
df_loos["avqz"] *= 37
display_formatted(df_exp)
df.to_csv("N_benchmark_converted.csv", index=False)

,state,exp1,exp2,theory
0,1E,292.30,293.41,296.74
1,3A2,222.74,NaN,NaN


In [19]:
df_casscf, dfdiff_casscf = get_state_values("NH3_casscf_states.csv")
df_casci, dfdiff_casci = get_state_values("NH3_casci_states.csv")

,state,avdz,avtz
0,gs,-56.534469(6),-56.56028(3)
1,1E,-56.2388(1),-56.252(2)
2,3A2,-56.300764(5),-56.32602(1)


,state,avdz,avtz
1,1E,+295.7(1),+308(2)
2,3A2,+233.704(7),+234.26(3)


,state,avdz,avtz
0,gs,-56.534702(5),-56.56139(1)
1,1E,-56.23023(5),-56.2550(2)
2,3A2,-56.301738(4),-56.32570(2)


,state,avdz,avtz
1,1E,+304.47(5),+306.4(2)
2,3A2,+232.963(6),+235.69(2)


In [20]:
# seem to struggle with singlets but not triplets
print("NH3 Loos")
display_formatted(df_loos)

print("NH3 CASSCF")
display_formatted(dfdiff_casscf)

print("NH3 CASCI")
display_formatted(dfdiff_casci)

print("NH3 Exp")
display_formatted(df_exp)

NH3 Loos


,state,avdz,avtz,avqz
0,1E,298.96,301.92,304.14
1,3A2,229.03,233.47,234.95


NH3 CASSCF


,state,avdz,avtz
1,1E,+295.7(1),+308(2)
2,3A2,+233.704(7),+234.26(3)


NH3 CASCI


,state,avdz,avtz
1,1E,+304.47(5),+306.4(2)
2,3A2,+232.963(6),+235.69(2)


NH3 Exp


,state,exp1,exp2,theory
0,1E,292.30,293.41,296.74
1,3A2,222.74,NaN,NaN
